In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_json('/home/pratik/Downloads/Frames-dataset/frames.json')

In [3]:
frames_dict = {'text': [], 'author': [], 'conversation_id': [], 'info': []}
for i in range(len(raw_data['turns'])):
    frame = raw_data['turns'].iloc[i]
    for each in frame:
        frames_dict['text'].append(each['text'])
        frames_dict['author'].append(each['author'])
        frames_dict['conversation_id'].append(i)
        #rint(len(each['labels']['frames']))
        temp = []
        for sen in each['labels']['frames']:
            temp.append(sen)
        frames_dict['info'].append(sen)

In [4]:
data = pd.DataFrame.from_dict(frames_dict)

In [5]:
rough_og = []
for line in data['info']:
    info = line['info']
    temp = []
    for each in info.get('or_city') or []:
        temp.append(each['val'])
    for each in info.get('dst_city') or []:
        temp.append(each['val'])
    rough_og.append(temp)

In [6]:
data['rough_og'] = rough_og

In [7]:
def get_og_text(text, original_text_list):
    temp = []
    for each in original_text_list:
        if each in text:
            temp.append(each)
    return temp

In [8]:
data['original_text'] = data.apply(lambda x: get_og_text(x['text'], x['rough_og']), axis=1)

In [9]:
data.head()

,author,conversation_id,info,text,rough_og,original_text
0,user,0,"{'info': {'intent': [{'val': 'book', 'negated'...",I'd like to book a trip to Atlantis from Capri...,"[Caprica, Atlantis]","[Caprica, Atlantis]"
1,wizard,0,"{'info': {'intent': [{'val': 'book', 'negated'...","Hi...I checked a few options for you, and unfo...","[Caprica, Atlantis]",[]
2,user,0,"{'info': {'intent': [{'val': 'book', 'negated'...","Yes, how about going to Neverland from Caprica...","[Caprica, Neverland]","[Caprica, Neverland]"
3,wizard,0,"{'info': {'intent': [{'val': 'book', 'negated'...",I checked the availability for this date and t...,"[Caprica, Neverland]",[]
4,user,0,"{'info': {'intent': [{'val': 'book', 'negated'...",I have no flexibility for dates... but I can l...,"[Atlantis, Atlantis]","[Atlantis, Atlantis]"


In [10]:
# raw_data = data

In [11]:
# raw_data.drop_duplicates(subset=['text'], inplace=True)
# len(raw_data)

In [12]:
# with open('gensim_embeddings_malubu.txt', 'a') as file_handler:
#     for body in raw_data['text']:
#         body = str(body)
#         file_handler.write(body)

In [13]:
# import gensim
# from gensim.models.keyedvectors import KeyedVectors
# sentences = gensim.models.word2vec.LineSentence('gensim_embeddings_malubu.txt')
# model = gensim.models.word2vec.Word2Vec(sentences,sample = 0.001, sg = 1, workers = 4, negative = 20, window = 10, seed = 1, alpha = 0.025, min_count = 0, min_alpha = 0.0001, size = 100)
# print("model")
# model.save('gensim_embeddings_malubu')


In [14]:
# word_vectors = gensim.models.Word2Vec.load('gensim_embeddings_malubu')

In [15]:
# word_vocab = word_vectors.wv.index2word

In [16]:
# word_vectors = word_vectors.wv.vectors

In [17]:
# import pickle
# file_handler = open('gensim_embeddings_malubu', 'wb')
# pickle.dump(word_vectors, file_handler)
# file_handler = open('gensim_embeddings_malubu_words', 'wb')
# pickle.dump(word_vocab, file_handler)

In [18]:
# file_handler = open('gensim_embeddings_malubu', 'rb')
# wordVectors = pickle.load(file_handler)
# file_handler = open('gensim_embeddings_malubu_words', 'rb')
# wordsList = pickle.load(file_handler)

In [19]:
# wordVectors.shape, len(wordsList)

In [20]:
# data['body'] = data['text']

In [21]:
# data = data[data['author'] == 'user'][['body', 'original_text']]

In [22]:
# data.to_csv('import pickle
# file_handler = open('gensim_embeddings_malubu', 'wb')
# pickle.dump(word_vectors, file_handler)
# file_handler = open('gensim_embeddings_malubu_words', 'wb')
# pickle.dump(word_vocab, file_handler)user_only.csv')

In [23]:
# data = pd.read_csv('user_only.csv')

In [24]:
context_dict = {'wizard': [], 'user': [], 'original_text': []}
for k in range(len(data['conversation_id'].unique())):
    x_dict = data[data['conversation_id'] == k].to_dict()
    for i in range(len(x_dict['author'])):

            if list(x_dict['author'].values())[i] == 'user':
                if i == 0:
                    context_dict['wizard'].append('No message')
                context_dict['user'].append(list(x_dict['text'].values())[i])
                context_dict['original_text'].append(list(x_dict['original_text'].values())[i])


            if list(x_dict['author'].values())[i] == 'wizard':
                if i == len(x_dict['author']) - 1:
                    context_dict['user'].append('No message')
                    context_dict['original_text'].append(list(x_dict['original_text'].values())[i])
                context_dict['wizard'].append(list(x_dict['text'].values())[i])

In [25]:
context_data = pd.DataFrame.from_dict(context_dict)

In [26]:
len([each for each in context_data['original_text'] if each])

3559

In [30]:
context_data['body'] = context_data['user']
context_data['context'] = context_data['wizard']

In [31]:
context_data.to_csv('context_data.csv')

In [33]:
pd.read_csv('context_data.csv')

,Unnamed: 0,original_text,user,wizard,body,context
0,0,"['Caprica', 'Atlantis']",I'd like to book a trip to Atlantis from Capri...,No message,I'd like to book a trip to Atlantis from Capri...,No message
1,1,"['Caprica', 'Neverland']","Yes, how about going to Neverland from Caprica...","Hi...I checked a few options for you, and unfo...","Yes, how about going to Neverland from Caprica...","Hi...I checked a few options for you, and unfo..."
2,2,"['Atlantis', 'Atlantis']",I have no flexibility for dates... but I can l...,I checked the availability for this date and t...,I have no flexibility for dates... but I can l...,I checked the availability for this date and t...
3,3,[],I suppose I'll speak with my husband to see if...,I checked the availability for that date and t...,I suppose I'll speak with my husband to see if...,I checked the availability for that date and t...
4,4,"['Gotham City', 'Mos Eisley']","Hello, I am looking to book a vacation from Go...",No message,"Hello, I am looking to book a vacation from Go...",No message
5,5,"['Gotham City', 'Neverland']",What about a trip from Gotham City to Neverlan...,"Hi. Sorry, I can't find any trips from Gotham ...",What about a trip from Gotham City to Neverlan...,"Hi. Sorry, I can't find any trips from Gotham ..."
6,6,['Mos Eisley'],Would any packages to Mos Eisley be available ...,"Sorry, I cannot find any trips leaving from Go...",Would any packages to Mos Eisley be available ...,"Sorry, I cannot find any trips leaving from Go..."
7,7,['Neverland'],"You know what, I'd like to try and visit Never...",There are no trips available to Mos Eisley.,"You know what, I'd like to try and visit Never...",There are no trips available to Mos Eisley.
8,8,"['Gotham City', 'Kobe']",Do you have any trips from Gotham City to Kobe...,I cannot find any trips available to Neverland.,Do you have any trips from Gotham City to Kobe...,I cannot find any trips available to Neverland.
9,9,['Birmingham'],"No, that's too far for me. I need a flight tha...",I can book you a 3 day trip to Kobe leaving fr...,"No, that's too far for me. I need a flight tha...",I can book you a 3 day trip to Kobe leaving fr...
